In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
neighborhoods.tail()

,Borough,Neighborhood,Latitude,Longitude
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631
305,Staten Island,Fox Hills,40.617311,-74.081740


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [16]:
CLIENT_ID = 'K0LBBCXQSY5SVWLURZPADRGUTWMJWQ4F10EI4JUPJ0ZAUE4M' # your Foursquare ID
CLIENT_SECRET = 'KWU4MKTWE1HDJRUFMNW0L4G2BZWMXTED2MNGAP2G5CVGIS5S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K0LBBCXQSY5SVWLURZPADRGUTWMJWQ4F10EI4JUPJ0ZAUE4M
CLIENT_SECRET:KWU4MKTWE1HDJRUFMNW0L4G2BZWMXTED2MNGAP2G5CVGIS5S


In [18]:
#manhattan_data.loc[0, 'Neighborhood'] #'Marble Hill'
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [17]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [19]:
# type your answer here

LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=K0LBBCXQSY5SVWLURZPADRGUTWMJWQ4F10EI4JUPJ0ZAUE4M&client_secret=KWU4MKTWE1HDJRUFMNW0L4G2BZWMXTED2MNGAP2G5CVGIS5S&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=2000&limit=500'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf06ef2db04f52f65b394ee'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4baf59e8f964a520a6f93be3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d102941735',
         'name': 'Yoga Studio',
         'pluralName': 'Yoga Studios',
         'primary': True,
         'shortName': 'Yoga Studio'}],
       'id': '4baf59e8f964a520a6f93be3',
       'location': {'address': '5500 Broadway',
        'cc': 'US',
        'city': 'Bronx',
        'country': 'United States',
        'crossStreet': '230th Street',
        'distance': 376,
        'formattedAddress': ['5500 Broadway (230th Street)',
         'Bronx, NY 10463',
         'United States'],
        'labe

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,postalCode
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204,10463
1,Tibbett Diner,Diner,40.880404,-73.908937,10463
2,Arturo's,Pizza Place,40.874412,-73.910271,10463
3,Sam's Pizza,Pizza Place,40.879435,-73.905859,10463
4,Loeser's Delicatessen,Sandwich Place,40.879242,-73.905471,10463


In [44]:
nearby_venues['categories'].value_counts()

Mexican Restaurant           7
Pizza Place                  7
Café                         5
Latin American Restaurant    4
Diner                        4
Park                         4
Coffee Shop                  4
Deli / Bodega                3
Bakery                       3
Bar                          3
Japanese Restaurant          2
Scenic Lookout               2
Trail                        2
Donut Shop                   2
Pub                          2
Spanish Restaurant           2
Burger Joint                 2
Yoga Studio                  2
Wine Bar                     2
Beer Bar                     1
Ice Cream Shop               1
Juice Bar                    1
Greek Restaurant             1
Venezuelan Restaurant        1
Department Store             1
History Museum               1
Tapas Restaurant             1
Frozen Yogurt Shop           1
Caribbean Restaurant         1
Warehouse Store              1
Supermarket                  1
Track                        1
Dessert 

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
 
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [27]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3324, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [51]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


In [28]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 332 uniques categories.


In [33]:
# get all of the Category 
manhattan_venues['Venue Category'].unique()

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Donut Shop', 'Coffee Shop',
       'Gym', 'Tennis Stadium', 'Seafood Restaurant', 'Department Store',
       'Video Game Store', 'Discount Store', 'Pharmacy', 'Supplement Shop',
       'Steakhouse', 'Ice Cream Shop', 'Bank', 'American Restaurant',
       'Bakery', 'Sandwich Place', 'Kids Store', 'Deli / Bodega', 'Hotel',
       'Greek Restaurant', 'General Entertainment', 'Cocktail Bar',
       'Chinese Restaurant', 'Spa', 'Museum', 'Bike Shop',
       'English Restaurant', 'Garden Center', 'Noodle House', 'Tea Room',
       'Salon / Barbershop', 'Indie Movie Theater',
       'New American Restaurant', 'Historic Site', 'Roof Deck',
       'Bubble Tea Shop', 'Hotpot Restaurant', 'Spanish Restaurant', 'Bar',
       'Sake Bar', 'Furniture / Home Store', 'Asian Restaurant',
       'Korean Restaurant', 'Malay Restaurant', 'Record Shop',
       'Organic Grocery', 'Italian Restaurant', 'Dim Sum Restaurant',
       'Cosmetics Shop', 'Vietnamese Re

In [29]:
# How many of them are related to resturant 
res = manhattan_venues[manhattan_venues['Venue Category'].str.contains('Restaurant')]

In [30]:
res['Venue Category'].unique()

array(['Seafood Restaurant', 'American Restaurant', 'Greek Restaurant',
       'Chinese Restaurant', 'English Restaurant',
       'New American Restaurant', 'Hotpot Restaurant',
       'Spanish Restaurant', 'Asian Restaurant', 'Korean Restaurant',
       'Malay Restaurant', 'Italian Restaurant', 'Dim Sum Restaurant',
       'Vietnamese Restaurant', 'Restaurant', 'Ramen Restaurant',
       'Dumpling Restaurant', 'Austrian Restaurant', 'Mexican Restaurant',
       'Cantonese Restaurant', 'Taiwanese Restaurant',
       'Vegetarian / Vegan Restaurant', 'Shanghai Restaurant',
       'Tapas Restaurant', 'Caribbean Restaurant', 'Indian Restaurant',
       'Latin American Restaurant', 'Sushi Restaurant', 'Thai Restaurant',
       'Arepa Restaurant', 'Fast Food Restaurant', 'Empanada Restaurant',
       'Japanese Restaurant', 'Mediterranean Restaurant',
       'Japanese Curry Restaurant', 'Cuban Restaurant',
       'French Restaurant', 'Ethiopian Restaurant', 'African Restaurant',
       'South

In [31]:
pizza= manhattan_venues[manhattan_venues['Venue Category'].str.contains('Pizza')]
#pizza.head(2)

In [32]:
pizza['Venue Category'].unique()

array(['Pizza Place'], dtype=object)

In [33]:
not_res = manhattan_venues[~manhattan_venues['Venue Category'].str.contains('Restaurant')]

In [34]:
not_res['Venue Category'].unique()

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Donut Shop', 'Coffee Shop',
       'Gym', 'Tennis Stadium', 'Department Store', 'Video Game Store',
       'Discount Store', 'Pharmacy', 'Supplement Shop', 'Steakhouse',
       'Ice Cream Shop', 'Bank', 'Bakery', 'Sandwich Place', 'Kids Store',
       'Clothing Store', 'Deli / Bodega', 'Hotel', 'General Entertainment',
       'Cocktail Bar', 'Spa', 'Museum', 'Bike Shop', 'Garden Center',
       'Noodle House', 'Tea Room', 'Salon / Barbershop',
       'Indie Movie Theater', 'Historic Site', 'Roof Deck',
       'Bubble Tea Shop', 'Bar', 'Sake Bar', 'Furniture / Home Store',
       'Record Shop', 'Organic Grocery', 'Cosmetics Shop', 'Music Venue',
       'Bookstore', 'Nail Salon', "Men's Store", 'Grocery Store',
       'Snack Place', 'Jewelry Store', 'Boutique',
       'Paper / Office Supplies Store', 'Optical Shop', 'Gift Shop',
       'Supermarket', 'Massage Studio', 'Shoe Store', 'Café',
       'Burger Joint', 'Market', 'Park', 'Liquor Sto

In [35]:
#allres = manhattan_venues[ manhattan_venues['Venue Category'].str.contains('Restaurant') || manhattan_venues['Venue Category'].str.contains('Pizza') ]

#df = pd.DataFrame({'col': ["apple is delicious",
#                           "banana is delicious","Orange","Stragnberry",
#                           "apple and banana both are delicious"]})
targets = ['Pizza', 'Restaurant','Diner','Steakhouse','Sandwich','Noodle House','Bistro','Veterinarian','Taco Place','Burrito Place','Salad Place']

#df[df.col.apply(lambda sentence: any(word in sentence for word in targets))]

allres=manhattan_venues[manhattan_venues['Venue Category'].apply(lambda sentence: any(word in sentence for word in targets))]

In [36]:
allres['Venue Category'].unique()

array(['Pizza Place', 'Diner', 'Seafood Restaurant', 'Steakhouse',
       'American Restaurant', 'Sandwich Place', 'Greek Restaurant',
       'Chinese Restaurant', 'English Restaurant', 'Noodle House',
       'New American Restaurant', 'Hotpot Restaurant',
       'Spanish Restaurant', 'Asian Restaurant', 'Korean Restaurant',
       'Malay Restaurant', 'Italian Restaurant', 'Dim Sum Restaurant',
       'Vietnamese Restaurant', 'Restaurant', 'Ramen Restaurant',
       'Dumpling Restaurant', 'Austrian Restaurant', 'Mexican Restaurant',
       'Cantonese Restaurant', 'Taiwanese Restaurant',
       'Vegetarian / Vegan Restaurant', 'Shanghai Restaurant',
       'Tapas Restaurant', 'Caribbean Restaurant', 'Indian Restaurant',
       'Latin American Restaurant', 'Sushi Restaurant', 'Thai Restaurant',
       'Arepa Restaurant', 'Bistro', 'Veterinarian',
       'Fast Food Restaurant', 'Empanada Restaurant',
       'Japanese Restaurant', 'Mediterranean Restaurant',
       'Japanese Curry Restaura

In [37]:
allres.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
7,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
14,Marble Hill,40.876551,-73.91066,Parrilla Latina,40.877473,-73.906073,Steakhouse
18,Marble Hill,40.876551,-73.91066,Boston Market,40.877430,-73.905412,American Restaurant


In [38]:
allres['Neighborhood'].value_counts()

Turtle Bay             49
Chinatown              48
Murray Hill            45
Greenwich Village      44
East Village           41
Noho                   39
Tudor City             38
Lenox Hill             38
Yorkville              37
West Village           37
Upper West Side        37
Sutton Place           35
Midtown South          35
Little Italy           33
Gramercy               32
Carnegie Hill          32
Financial District     32
Tribeca                31
Flatiron               30
Civic Center           30
Upper East Side        29
Midtown                29
Clinton                28
Chelsea                28
Washington Heights     27
Manhattan Valley       27
Hamilton Heights       25
Soho                   23
Inwood                 22
Hudson Yards           21
Lower East Side        21
Manhattanville         18
Central Harlem         18
East Harlem            17
Battery Park City      17
Lincoln Square         16
Morningside Heights    11
Marble Hill             7
Roosevelt Is

In [39]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Sm

In [40]:
manhattan_onehot.shape

(3324, 333)

In [41]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Sm

In [42]:
manhattan_grouped.shape

(40, 333)

In [43]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.07
2               Hotel  0.05
3       Memorial Site  0.04
4  Italian Restaurant  0.03


----Carnegie Hill----
         venue  freq
0  Pizza Place  0.06
1  Coffee Shop  0.06
2         Café  0.04
3  Yoga Studio  0.03
4          Gym  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.06
1   Chinese Restaurant  0.06
2  Fried Chicken Joint  0.04
3           Public Art  0.04
4   Seafood Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1   Italian Restaurant  0.05
2       Ice Cream Shop  0.05
3               Bakery  0.04
4  American Restaurant  0.04


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.10
1  American Restaurant  0.04
2         Cocktail Bar  0.04
3      Bubble Tea Shop  0.03
4  Dumpling Restaurant  0.03


----Civic Center----
                  venue  freq
0    Italian Resta

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Wine Shop,Italian Restaurant,Gym,Clothing Store,Plaza,Burger Joint
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Wine Shop,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant
2,Central Harlem,African Restaurant,Chinese Restaurant,Public Art,Fried Chicken Joint,Seafood Restaurant,American Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,French Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Seafood Restaurant,Theater,Hotel,Gift Shop
4,Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Dumpling Restaurant,Bubble Tea Shop,Spa,Vietnamese Restaurant,Asian Restaurant,Jewelry Store,Bakery


In [46]:
neighborhoods_venues_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Upper East Side,Exhibit,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Juice Bar,Bakery,Art Gallery,French Restaurant,Cocktail Bar,Spa
36,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Mediterranean Restaurant,Bakery,Indian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Pub
37,Washington Heights,Café,Mobile Phone Shop,Bakery,Mexican Restaurant,Pizza Place,Sandwich Place,Grocery Store,New American Restaurant,Supplement Shop,Gym
38,West Village,Italian Restaurant,New American Restaurant,Cosmetics Shop,Park,Jazz Club,Wine Bar,American Restaurant,Coffee Shop,Bakery,Gastropub
39,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Sushi Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,Deli / Bodega,Diner


In [48]:
allres.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner


In [52]:
# create map of Manhattan using latitude and longitude values
map_allres = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(allres['Venue Latitude'], allres['Venue Longitude'], allres['Venue Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_allres)  
    
# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_allres)       
    
map_allres